<a href="https://colab.research.google.com/github/naveenbanty/Object-detection-/blob/main/Object_detection_of_webmarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Downloading and preprocessing on Dataset

In [2]:
import requests
from tqdm import tqdm
BASE_URL = 'http://yuhang.rsise.anu.edu.au/download/'

In [3]:
list_links = []
for p in range(1, 11):
  list_links.append(f'db{p}.zip')

In [ ]:
for link in list_links:
  url = BASE_URL + link
  !wget {url}

--2021-04-23 02:07:05--  http://yuhang.rsise.anu.edu.au/download/db1.zip
Resolving yuhang.rsise.anu.edu.au (yuhang.rsise.anu.edu.au)... 150.203.208.64
Connecting to yuhang.rsise.anu.edu.au (yuhang.rsise.anu.edu.au)|150.203.208.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 192894514 (184M) [application/zip]
Saving to: ‘db1.zip’

db1.zip             100%[===================>] 183.96M  7.36MB/s    in 31s     

2021-04-23 02:07:37 (5.98 MB/s) - ‘db1.zip’ saved [192894514/192894514]

--2021-04-23 02:07:37--  http://yuhang.rsise.anu.edu.au/download/db2.zip
Resolving yuhang.rsise.anu.edu.au (yuhang.rsise.anu.edu.au)... 150.203.208.64
Connecting to yuhang.rsise.anu.edu.au (yuhang.rsise.anu.edu.au)|150.203.208.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203405885 (194M) [application/zip]
Saving to: ‘db2.zip’

db2.zip             100%[===================>] 193.98M  7.55MB/s    in 32s     

2021-04-23 02:08:11 (6.06 MB/s) - ‘db2.zip’ s

In [ ]:
!wget https://raw.githubusercontent.com/ParallelDots/generic-sku-detection-benchmark/master/annotations/WebMarket/WebMarket_coco_gt_object.csv

In [ ]:
!ls

In [ ]:
for link in list_links:
  !unzip {link}

In [ ]:
for link in list_links:
  !rm -r {link}

In [ ]:
!ls

In [ ]:
#Installing Detectron2 and dependencies

In [ ]:
!pip install pyyaml==5.1

In [ ]:
!pip install pycocotools>=2.0.1

In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

In [ ]:
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import glob
from google.colab.patches import cv2_imshow
import os
import ntpath
import numpy as np
import cv2
import random
import itertools
import pandas as pd
from tqdm import tqdm
import urllib
import json
import PIL.Image as Image
import tensorboard

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8


In [ ]:
folders = []
for p in range(1, 11):
  folders.append(f'db{p}')

In [ ]:
!ls

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

In [ ]:
annote = pd.read_csv('WebMarket_coco_gt_object.csv')

In [ ]:
annote.head()

In [ ]:
annote.info()

# **Preparing datasets**

In [ ]:
dataset = pd.DataFrame()
for dir in list_links:
  dir_name = dir.split('.')[0]
  for img_name in os.listdir(dir_name):
    temp_df = annote[annote['image_name'] == img_name]
    temp_df['file_path'] = os.path.join(dir_name, img_name)
    dataset = pd.concat([dataset, temp_df])

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
os.makedirs('objects', exist_ok = True)

In [ ]:
unique_img = dataset.image_name.unique()

In [ ]:
dataset = dataset.rename(columns=  {'class' : 'class_name'})

In [ ]:
train_files = set(np.random.choice(unique_img, int(len(unique_img) * 0.90), replace=False))
train_df = dataset[dataset.image_name.isin(train_files)]
test_df = dataset[~dataset.image_name.isin(train_files)]

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
classes = dataset.class_name.unique().tolist()

In [ ]:
def create_dataset_dicts(df, classes):
  dataset_dicts = []
  for image_id, img_name in enumerate(df.image_name.unique()):

    record = {}

    image_df = df[df.image_name == img_name]

    record["file_name"] = image_df.file_path.values[0]
    record["image_id"] = image_id
    record["height"], record["width"] = cv2.imread(image_df.file_path.values[0]).shape[:2]

    objs = []
    for _, row in image_df.iterrows():

      x1 = int(row.x1)
      y1 = int(row.y1)
      x2 = int(row.x2)
      y2 = int(row.y2)

      poly = [
          (x1, y1), (x2, y1), 
          (x2, y2), (x1, y2)
      ]
      poly = list(itertools.chain.from_iterable(poly))

      obj = {
        "bbox": [x1, y1, x2, y2],
        "bbox_mode": BoxMode.XYXY_ABS,
        "segmentation": [poly],
        "category_id": classes.index(row.class_name),
      }
      objs.append(obj)

    record["annotations"] = objs
    dataset_dicts.append(record)
  return dataset_dicts

In [ ]:
for d in ["train", "val"]:
  DatasetCatalog.register("objects_" + d, lambda d=d: create_dataset_dicts(train_df if d == "train" else test_df, classes))
  MetadataCatalog.get("objects_" + d).set(thing_classes=classes)

statement_metadata = MetadataCatalog.get("objects_train")

# **Visualizing training dataset¶**

In [ ]:
dataset_dicts = create_dataset_dicts(train_df[:300], classes)
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=statement_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
class CocoTrainer(DefaultTrainer):
  
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

# **Model building and Training**

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("objects_train",)
cfg.DATASETS.TEST = ("objects_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001 
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 2000   
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  
cfg.TEST.EVAL_PERIOD = 500
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output

# **Predicting on Test dataset.**

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

# **Visualizing Test results.**

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = create_dataset_dicts(test_df, classes)
for d in random.sample(dataset_dicts, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  
    v = Visualizer(im[:, :, ::-1],
                   metadata=statement_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

# **Inference on test set and storing the result**

In [ ]:
evaluator = COCOEvaluator("objects_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "objects_val")
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
os.makedirs("annotated_results", exist_ok=True)
test_image_paths = test_df.file_path.unique()

In [ ]:
for test_image in test_image_paths:
  im = cv2.imread(test_image)
  outputs = predictor(im)
  v = Visualizer(
    im[:, :, ::-1],
    metadata=statement_metadata, 
    scale=0.5, 
    instance_mode=ColorMode.IMAGE_BW
  )
  instances = outputs["instances"].to("cpu")
  v = v.draw_instance_predictions(instances)
  result = v.get_image()[:, :, ::-1]
  file_name = ntpath.basename(test_image)
  write_res = cv2.imwrite(f'annotated_results/{file_name}', result)

In [ ]:
!ls annotated_results

In [ ]:
annotated_images = [f'annotated_results/{f}' for f in test_df.image_name.unique()]

In [ ]:
img = cv2.imread(annotated_images[67])

plt.imshow(img)
plt.axis('off');

In [ ]:
random_sample = random.sample(annotated_images, 1)
img = cv2.imread(random_sample[0])

plt.imshow(img)
plt.axis('off');

In [ ]:
# is there any more code to paste?
# No anna 
# ok, 
# let's do a full run then 
# oka